In [12]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#from scipy.stats import norm
#from scipy.stats import linregress
#import statsmodels.formula.api as smf
import csv
import scipy.optimize as opt
import pandas_datareader as pdr
from datetime import datetime


In [2]:
# function to read in data from a csv
def readPricesCSV(file):
    df = pd.read_csv(file)
    dfPrices = df["Close"]
    return dfPrices

In [17]:
# function to read in stock prices from yahoo finance in real time
def readPricesYF(start_date, end_date, tickers):
    stockPrices = pdr.get_data_yahoo(tickers, start=start_date, end=end_date)   # read in the data
    stockPrices = stockPrices.filter(like='Adj Close')   # tickers are columns and data values are adjusted closing prices with the date as the index
    stockPrices.columns = tickers   # change column names to be their tickers
    stockPrices = stockPrices.stack().swaplevel().sort_index().reset_index()    # make the data tall and clean it up so it is easier to perform analysis on
    stockPrices.columns = ['Firm','Date','Adj Close']     # rename the columns
#    stockPrices['Return'] = stockPrices.groupby('Firm')['Adj Close'].pct_change()    # get the daily returns for each ticker
    return stockPrices


In [16]:
# the objective function
def obj(weights, means, risk_level, covMatr):
    
    #get the return measure given by the weights and mean
    for weight,mean in weights,means:
        sum_return += weight*mean
    num_stocks = len(means)
    
    #get the risk measure given by the weights
    risk_measure = np.matmul(np.matmul(np.transpose(np.array(weights)), covMatr), np.array(weights))

    #return the objective function value
    return -(sum_return - (risk_level * risk_measure))


In [18]:
#choose which stocks to include in the optimization of the portfolio (list the tickers)
tickers = ['AAPL', 'MSFT', 'NFLX', 'AMZN']

#set the start and end dates- choose last six months in this case
start_date = datetime(2020, 8, 25) #year, month, day
end_date = datetime(2021,2,25) #year, month, day

stockPrices = readPricesYF(start_date, end_date, tickers)    # use the function defined above to get data from Yahoo Finance


In [19]:
stockPrices

,Firm,Date,Adj Close
0,AAPL,2020-08-25,124.424088
1,AAPL,2020-08-26,126.116135
2,AAPL,2020-08-27,124.608498
3,AAPL,2020-08-28,124.406647
4,AAPL,2020-08-31,128.625549
...,...,...,...
503,NFLX,2021-02-19,540.219971
504,NFLX,2021-02-22,533.780029
505,NFLX,2021-02-23,546.150024
506,NFLX,2021-02-24,553.409973


In [125]:
# get the optimization ready

num_stocks = len(tickers)
stock_prices_format = stockPrices.set_index(['Date', 'Firm']).unstack()  #.T
stock_prices_format.reset_index()
#stock_prices_format.iloc[:,[0,1,2,3]]
#stock_prices_format.iloc[:,[0]]
#prices = stock_prices_format.iloc[:,[0,1,2,3]]
prices = stockPrices.reset_index().drop(['Date','index'], axis=1).set_index('Firm').unstack()   #.drop('index', axis=1)
prices

#prices = stock_prices_format.drop('Date', axis=1)

#cov_matr = np.cov(np.array([stock_prices_format.iloc[:,[0,1,2,3]]]))


           Firm
Adj Close  AAPL    124.424088
           AAPL    126.116135
           AAPL    124.608498
           AAPL    124.406647
           AAPL    128.625549
                      ...    
           NFLX    540.219971
           NFLX    533.780029
           NFLX    546.150024
           NFLX    553.409973
           NFLX    546.700012
Length: 508, dtype: float64